In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import local_parameters
import urllib.request 
import os

In [36]:
scrape_html = False
parse_html = False

## Scrape HTML

In [3]:
%%time

if scrape_html:
    cookie = JSESSIONID=''
    headers = {
        'User-Agent': local_parameters.UserAgent,
        'Cookie' : local_parameters.Cookie}
    session = requests.session()

    test=False
    issueListFilename = "2001-2008URLs.csv"
    _source_html_path = 'source_html2'
    get_year = 2003
    count = 78
    first = 1

    def save_html(year,source_html): 
        global count, first, my_year
        if first:
            first = 0
            my_year = year
            count += 1
        else:
            if my_year == year:
                count += 1
            else: 
                my_year = year
                count = 1

        print(str(year) + '-' + str(count) + '.html')
        f = open(os.path.join(_source_html_path, str(year) + '-' + str(count) + '.html'), 'w')
        f.write(source_html)
        f.close

    def get_url_source_html(url):
        response = session.get(str(url), headers=headers)
        redirected_url = response.history[-1].url
        print(redirected_url)
        response = session.get(redirected_url, headers=headers)
        return response.text

    def process_url(row):
        print(str(count) + ": " + row.url)
        source_html = get_url_source_html(row.url)
        #time.sleep(5)
        save_html(row.year, source_html)
        #time.sleep(10)


    if test == False:
        df = pd.read_csv(issueListFilename)
        #df = df[df.year == get_year] 
        for index, row in df.iterrows():
           process_url(row)

    else:
        #url = 'https://doi.org/10.1093/geront/gnx189'
        year = '0000'
        url = 'https://doi.org/10.1093/geront/43.2.192'
        data = [[year,url]]
        df = pd.DataFrame(data,columns=['year','url'])

        process_url(df.url.iloc[0])


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


## Load HTML, Parse, Save

In [32]:
def parse_source_html(filename, source_html):
    soup = BeautifulSoup(source_html, "html.parser")

    #raw_html.append(soup.get_text())
    
    #citation_date = soup.find("div", {"class": "citation-date"}).text
    #citation_dates.append(citation_date)

    filename = os.path.basename(filename)
    abstract = soup.find(attrs={'class': 'abstract'})#.text
    if abstract:
        abstract = abstract.text
    parsed_data.append(pd.Series([filename, abstract], index=columns))



In [33]:
%%time

if parse_html:
    import sys
    import glob
    import errno
    counter = 0
    columns = ['File', 'Abstract']
    parsed_data = []
    path = '/Users/bbarsness/code/GerontologyML/source_html/*.html'  
    #path = '/Users/bbarsness/code/GerontologyML/source_html/2003-32.html'
    files = glob.glob(path)   
    for filename in files: 
        try:
            with open(filename) as f: 
                #sys.stdout.write(f.read())
                parse_source_html(filename, f.read())
                counter += 1; print(str(counter) + ' ', end='')
        except IOError as exc:
            if exc.errno != errno.EISDIR: # Do not fail if a directory is found, just ignore it.
                raise # Propagate other kinds of IOError.

    df = pd.DataFrame(parsed_data, columns = columns) 
    df.to_excel('parsed_data.xlsx')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [34]:
df.head()

,File,Abstract
0,2008-84.html,Purpose: Researchers' knowledge regarding olde...
1,2006-75.html,Purpose: Individual expectations among communi...
2,2004-48.html,Purpose: Turnover in nursing homes is a widesp...
3,2002-36.html,Case studies are used extensively in gerontolo...
4,2005-5.html,Purpose: We first describe which states have p...


In [65]:
import spacy

nlp = spacy.load('en_core_web_lg')

# create a list of stop words
from spacy.lang.en.stop_words import STOP_WORDS
common_terms = list(STOP_WORDS)

In [62]:
df = pd.read_excel('parsed_data.xlsx')
df = df.dropna()
text = [text for text in df['Abstract']]
len(text)

701

In [77]:
text = ' '.join(text)
text

'Purpose:\u2002Researchers\' knowledge regarding older users of illicit drugs is limited despite the increasing numbers of users. In this article, we apply a life course perspective to gain a further understanding of older adult drug use, specifically contrasting early- and late-onset heroin users.\u2003Design and Methods:We collected qualitative data from 29 older heroin users. Life course analysis focused on the users\' experiences across the life span.\u2003Results:\u2002\u2002The findings suggest that those aging into heroin use (late onset) are disadvantaged compared to those who are maturing in (early onset) except in areas of health.\u2003Implications:\u2002We propose that conceptualizing the use of heroin and other illicit drugs among older adults based on the user\'s life course trajectory will provide insights for social and health services, including drug treatment. Purpose:\u2002Individual expectations among community-dwelling older adults and their subsequent effect on pla

In [78]:
doc = nlp(text)

In [79]:
print(doc[0:10])

Purpose: Researchers' knowledge regarding older users of


In [81]:

from gensim.sklearn_api.phrases import PhrasesTransformer

sklearn_phrases = PhrasesTransformer(min_count=5, threshold=5)
sklearn_phrases.fit(text)

PhrasesTransformer(delimiter=b'_', max_vocab_size=40000000, min_count=5,
          progress_per=10000, scoring='default', threshold=5)

In [74]:

# review phrase matches
phrases = []
for terms in sklearn_phrases.transform(text):
    for term in terms:
        if term.count('_') >= 1:
            phrases.append(term)

print(set(phrases))

set()


/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [63]:
from gensim.models.phrases import Phrases, Phraser

In [66]:
phrases = Phrases(
      text
    , common_terms=common_terms
    , min_count=5
    , threshold=5
    , scoring='default'
)

phrases

In [67]:
bigram = Phraser(phrases)

bigram

In [70]:
def print_phrases(phraser, text_stream, num_underscores=1):
    """ identify phrases from a text stream by searching for terms that
        are separated by underscores and include at least num_underscores
    """
    
    phrases = []
    for terms in phraser[text_stream]:
        for term in terms:
            if term.count('_') >= num_underscores:
                phrases.append(term)
    print(set(phrases))

In [71]:

print_phrases(bigram, text)

set()


In [53]:
for doc in nlp.pipe(df.Abstract.head(10), disable=['parser','tagger','ner']):
    print(doc)
    print()

Purpose: Researchers' knowledge regarding older users of illicit drugs is limited despite the increasing numbers of users. In this article, we apply a life course perspective to gain a further understanding of older adult drug use, specifically contrasting early- and late-onset heroin users. Design and Methods:We collected qualitative data from 29 older heroin users. Life course analysis focused on the users' experiences across the life span. Results:  The findings suggest that those aging into heroin use (late onset) are disadvantaged compared to those who are maturing in (early onset) except in areas of health. Implications: We propose that conceptualizing the use of heroin and other illicit drugs among older adults based on the user's life course trajectory will provide insights for social and health services, including drug treatment.

Purpose: Individual expectations among community-dwelling older adults and their subsequent effect on placement status have recently been considered

In [57]:

from gensim.sklearn_api.phrases import PhrasesTransformer

sklearn_phrases = PhrasesTransformer(min_count=5, threshold=5)
sklearn_phrases.fit(matched_sents)


1344233

In [50]:
df.iloc[8]

File        2005-140.html
Abstract              NaN
Name: 8, dtype: object

In [ ]:
def get_soup(myarticle):
    return BeautifulSoup(myarticle, "lxml")

def strip_hyperlinks(mysoup):
    for a in mysoup.findAll('a'):
        a.replaceWith(" %s " % a.string)
    return(mysoup)

def strip_references(mysoup):
    for div in mysoup.find_all("div", {'class':'ref-list'}): 
        div.decompose()
    return(mysoup)

def strip_copywrite(mysoup):
    for div in mysoup.find_all("div", {'class':'copyright copyright-statement'}): 
        div.decompose()
    return(mysoup)

def strip_tables(mysoup):
    codetags = mysoup.find_all('table')
    for codetag in codetags:
       codetag.extract()
    return(mysoup)

def strip_rnt(mysoup):
    return(mysoup.get_text().replace('\r', ' ').replace('\n', ' ').replace('\t', ' '))
 
def clean_article(mysoup):
    #soup = get_soup(myarticle)
    mysoup = strip_hyperlinks(mysoup)
    mysoup = strip_references(mysoup)
    mysoup = strip_copywrite(mysoup)
    mysoup = strip_tables(mysoup)
    soup_text = strip_rnt(mysoup)
    return soup_text


In [ ]:
def parse_source_html(source_html):
    soup = BeautifulSoup(source_html, "html.parser")

    raw_html.append(soup.get_text())
    
    citation_date = soup.find("div", {"class": "citation-date"}).text
    citation_dates.append(citation_date)

    citation = soup.find(attrs={'class': 'ww-citation-primary'}).text
    citations.append(citation)

    article_body = soup.find("div", {"class": "article-body"})
    articles.append(article_body)
    
    types.append(soup.find(attrs={'property': 'og:type'}).attrs['content'])
    titles.append(soup.find(attrs={'property': 'og:title'}).attrs['content'])
    descriptions.append(soup.find(attrs={'property': 'og:description'}).attrs['content'])
    urls.append(soup.find(attrs={'property': 'og:url'}).attrs['content'])
    updated_times.append(soup.find(attrs={'property': 'og:updated_time'}).attrs['content'])
    site_names.append(soup.find(attrs={'property': 'og:site_name'}).attrs['content'])
    
    article = clean_article(article_body)
    clean_articles.append(article)
    

In [ ]:
%%time

test=False
citations = []
citation_dates = []
articles = []
clean_articles = []
types = []
titles = []
descriptions = []
urls = []
updated_times = []
site_names = []
raw_html = []

issueListFilename = "2001-2008URLs.csv"
get_year = 2003
count = 0

def process_url(my_url):
    global count
    print(str(count) + ": " + my_url)
    source_html = get_url_source_html(my_url)
    #time.sleep(5)
    #print(source_html)
    parse_source_html(source_html)
    #time.sleep(10)
    count += 1
         
if test == False:
    df = pd.read_csv(issueListFilename)
    df = df[df.year == get_year] 
    print(len(df.index))
    df.url.apply(lambda x: process_url(x))
    
else:
    #url = 'https://doi.org/10.1093/geront/gnx189'
    url = 'https://doi.org/10.1093/geront/43.2.192'
    print(url)
    source_html = get_url_source_html(url)
    parse_source_html(source_html)


In [ ]:
df = pd.DataFrame({'citation_date': citation_dates,
                    'citation': citations,
                    'title': titles,
                    'type': types,
                    'description': descriptions,
                    'url': urls,
                    'updated_time': updated_times,
                    'site_name': site_names,
                    'article': articles,
                    'clean_article': clean_articles,
                    'raw_html' : raw_html
                    },columns = ['citation_date', 'citation', 'title', 'type', 'description', 'url', 'updated_time', 'site_name', 'article', 'clean_article', 'raw_html'])
print(df.info())
df.head(10)

In [ ]:
df["clean_article"][0]

In [ ]:
count = 0
def my_print(my_x):
    global count
    print(count, len(my_x))
    count += 1

df["clean_article"].apply(lambda x: my_print(x))

In [ ]:
#df['clean_article'].loc[129]
df.loc[129]

In [ ]:
count = 0
remove_indices = []
def get_short_articles(my_x):
    global remove_indices, count
    if len(my_x) < 500:
        remove_indices.append(count)
    count += 1

df["clean_article"].apply(lambda x: get_short_articles(x))
print("Initial set: ", len(df))
print("Remove set: " + str(len(remove_indices)))
print(remove_indices)

df_reduced = df.drop(remove_indices)
print("After removal: " + str(len(df_reduced)))

In [ ]:
count = 0
def my_print(my_x):
    global count
    print(count, len(my_x))
    count += 1

df_reduced["clean_article"].apply(lambda x: my_print(x))

In [ ]:
import os
base_filename = 'articles' + str(get_year) + '.txt'
with open(os.path.join(base_filename),'w') as outfile:
    df_reduced["clean_article"].apply(lambda x: outfile.write(x + '\n'))
outfile.close()
num_lines = sum(1 for line in open(base_filename))
print(num_lines)
            

In [ ]:
import sys
sys.setrecursionlimit(10000)
pickle_file = 'data_' + str(get_year) + '.pkl.gzip'
df_reduced.to_pickle(pickle_file, compression='gzip')

In [ ]:
result_filename = 'articles' + str(get_year) + '.csv'
df_reduced.to_csv(result_filename)